In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ydata_profiling import ProfileReport
from movie_recommender_system import paths
import pandas as pd

In [3]:
movie_data_path = paths.data_raw_dir() / "tmdb_5000_movie_dataset/movies.csv"
credits_data_path = paths.data_raw_dir() / "tmdb_5000_movie_dataset/credits.csv"

In [4]:
df_movies = pd.read_csv(movie_data_path)
profile_movies = ProfileReport(df_movies, title="Profiling Report")
#profile_movies.to_file(paths.reports_dir() / "00_raw_movies.html")

In [5]:
df_credits = pd.read_csv(credits_data_path)
profile_credits = ProfileReport(df_credits, title="Profiling Report")
#profile_credits.to_file(paths.reports_dir() / "00_raw_credits.html")

In [6]:
df_dataset = df_movies.merge(df_credits, on="title")

In [7]:
list(df_dataset.columns)

['budget',
 'genres',
 'homepage',
 'id',
 'keywords',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'vote_average',
 'vote_count',
 'movie_id',
 'cast',
 'crew']

In [8]:
df_dataset = df_dataset[['movie_id','title','overview','genres','keywords','cast','crew']]

In [9]:
df_dataset.sample(5, random_state=1)

,movie_id,title,overview,genres,keywords,cast,crew
1296,8842,Against the Ropes,A fictional story inspired by North America's ...,"[{""id"": 10749, ""name"": ""Romance""}, {""id"": 18, ...","[{""id"": 5565, ""name"": ""biography""}, {""id"": 607...","[{""cast_id"": 1, ""character"": ""Jackie Kallen"", ...","[{""credit_id"": ""555f44e69251417e51002c21"", ""de..."
4023,50538,Evil Words,"One day, for no apparent reason, a cop kills 1...","[{""id"": 27, ""name"": ""Horror""}, {""id"": 9648, ""n...","[{""id"": 9937, ""name"": ""suspense""}, {""id"": 1002...","[{""cast_id"": 1, ""character"": ""Thomas Roy"", ""cr...","[{""credit_id"": ""58ade6cec3a36861d60004cb"", ""de..."
1653,28902,In Dreams,Claire Cooper dreams strange things from time ...,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 4, ""character"": ""Claire Cooper"", ...","[{""credit_id"": ""55970f30c3a36855e300026d"", ""de..."
4060,309425,Do You Believe?,When a pastor is shaken by the visible faith o...,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 18, ""na...","[{""id"": 5950, ""name"": ""christian""}]","[{""cast_id"": 0, ""character"": ""Samantha"", ""cred...","[{""credit_id"": ""59a42566925141775b05a72b"", ""de..."
4735,272726,Dude Where's My Dog?,"Left home alone with his dog Harry, young Ray ...","[{""id"": 10751, ""name"": ""Family""}]",[],"[{""cast_id"": 6, ""character"": """", ""credit_id"": ...","[{""credit_id"": ""592bed26c3a36877bc07a7e0"", ""de..."


In [10]:
df_dataset.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [11]:
df_dataset_valid = df_dataset.dropna()

In [12]:
import ast

def extract_data_from_dict_string(string_dict, key) -> list:
    value_list = []
    for item in ast.literal_eval(string_dict):
        value_list.append(item[key])
    return value_list

In [13]:
df_dataset_valid.loc[:, 'genres'] = df_dataset_valid['genres'].apply(lambda x: extract_data_from_dict_string(x, 'name'))
df_dataset_valid.loc[:, 'keywords'] = df_dataset_valid['keywords'].apply(lambda x: extract_data_from_dict_string(x, 'name'))


In [14]:
df_dataset_valid.sample(5, random_state=1)

,movie_id,title,overview,genres,keywords,cast,crew
3229,20360,Namastey London,Indian-born Manmohan Malhotra decided to re-lo...,"[Drama, Foreign, Romance]",[],"[{""cast_id"": 1, ""character"": ""Arjun Ballu Sing...","[{""credit_id"": ""52fe43e3c3a368484e00404f"", ""de..."
4143,115210,Stitches,"The clumsy and unfunny clown Richard ""Stitches...",[Horror],"[slapstick, teen angst, slaughter, dark comedy...","[{""cast_id"": 2, ""character"": ""Stitches"", ""cred...","[{""credit_id"": ""551becafc3a368766c0032c6"", ""de..."
4694,13363,The Man from Earth,An impromptu goodbye party for Professor John ...,"[Science Fiction, Drama]","[philosophy, secret, birthday, professor, psyc...","[{""cast_id"": 3, ""character"": ""John Oldman"", ""c...","[{""credit_id"": ""52fe45609251416c75054411"", ""de..."
3683,64720,Take Shelter,"Plagued by a series of apocalyptic visions, a ...","[Thriller, Drama]","[shelter, end of the world, sanity, apocalypse]","[{""cast_id"": 3, ""character"": ""Curtis LaForche""...","[{""credit_id"": ""5910dc799251414edb0380ec"", ""de..."
1891,9469,He Got Game,A basketball player's father must try to convi...,[Drama],"[prison, father son relationship, homicide, be...","[{""cast_id"": 8, ""character"": ""Jake Shuttleswor...","[{""credit_id"": ""5989db109251414bd1016292"", ""de..."


In [15]:
ast.literal_eval(df_dataset_valid["cast"][0])[0]

{'cast_id': 242,
 'character': 'Jake Sully',
 'credit_id': '5602a8a7c3a3685532001c9a',
 'gender': 2,
 'id': 65731,
 'name': 'Sam Worthington',
 'order': 0}

In [16]:
ast.literal_eval(df_dataset_valid["crew"][0])

[{'credit_id': '52fe48009251416c750aca23',
  'department': 'Editing',
  'gender': 0,
  'id': 1721,
  'job': 'Editor',
  'name': 'Stephen E. Rivkin'},
 {'credit_id': '539c47ecc3a36810e3001f87',
  'department': 'Art',
  'gender': 2,
  'id': 496,
  'job': 'Production Design',
  'name': 'Rick Carter'},
 {'credit_id': '54491c89c3a3680fb4001cf7',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Sound Designer',
  'name': 'Christopher Boyes'},
 {'credit_id': '54491cb70e0a267480001bd0',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Supervising Sound Editor',
  'name': 'Christopher Boyes'},
 {'credit_id': '539c4a4cc3a36810c9002101',
  'department': 'Production',
  'gender': 1,
  'id': 1262,
  'job': 'Casting',
  'name': 'Mali Finn'},
 {'credit_id': '5544ee3b925141499f0008fc',
  'department': 'Sound',
  'gender': 2,
  'id': 1729,
  'job': 'Original Music Composer',
  'name': 'James Horner'},
 {'credit_id': '52fe48009251416c750ac9c3',
  'department': 'Directing',
  

In [17]:
def extract_cast(string_dict: str, key: str, count: int) -> list:
    value_list = []
    counter = 0
    for item in ast.literal_eval(string_dict):
        if counter == count:
            break
        else:
            value_list.append(item[key])
            counter += 1
    return value_list

In [18]:
cast_count = 3
df_dataset_valid.loc[:, 'cast'] = df_dataset_valid['cast'].apply(lambda x: extract_cast(x, 'name', cast_count))

In [19]:
def extract_crew(string_dict: str, key: str, crew_jobs: list) -> list:
    value_list = []
    for item in ast.literal_eval(string_dict):
        if item['job'] in crew_jobs:
            value_list.append(item[key])
    return value_list

In [20]:
crew_jobs_list = ['Director']
df_dataset_valid.loc[:, 'crew'] = df_dataset_valid['crew'].apply(lambda x: extract_crew(x, 'name', crew_jobs_list))

In [21]:
df_dataset_valid.sample(5, random_state=1)

,movie_id,title,overview,genres,keywords,cast,crew
3229,20360,Namastey London,Indian-born Manmohan Malhotra decided to re-lo...,"[Drama, Foreign, Romance]",[],"[Akshay Kumar, Katrina Kaif, Rishi Kapoor]",[Vipul Amrutlal Shah]
4143,115210,Stitches,"The clumsy and unfunny clown Richard ""Stitches...",[Horror],"[slapstick, teen angst, slaughter, dark comedy...","[Ross Noble, Tommy Knight, Gemma-Leah Devereux]",[Conor McMahon]
4694,13363,The Man from Earth,An impromptu goodbye party for Professor John ...,"[Science Fiction, Drama]","[philosophy, secret, birthday, professor, psyc...","[David Lee Smith, Tony Todd, John Billingsley]",[Richard Schenkman]
3683,64720,Take Shelter,"Plagued by a series of apocalyptic visions, a ...","[Thriller, Drama]","[shelter, end of the world, sanity, apocalypse]","[Michael Shannon, Jessica Chastain, Katy Mixon]",[Jeff Nichols]
1891,9469,He Got Game,A basketball player's father must try to convi...,[Drama],"[prison, father son relationship, homicide, be...","[Denzel Washington, Ray Allen, Milla Jovovich]",[Spike Lee]


In [22]:
def remove_space(string_list: list) -> list:
    return [x.replace(" ", "") for x in string_list]

df_dataset_valid.loc[:, 'genres'] = df_dataset_valid['genres'].apply(lambda x: remove_space(x))

In [23]:
df_dataset_valid.sample(5, random_state=1)

,movie_id,title,overview,genres,keywords,cast,crew
3229,20360,Namastey London,Indian-born Manmohan Malhotra decided to re-lo...,"[Drama, Foreign, Romance]",[],"[Akshay Kumar, Katrina Kaif, Rishi Kapoor]",[Vipul Amrutlal Shah]
4143,115210,Stitches,"The clumsy and unfunny clown Richard ""Stitches...",[Horror],"[slapstick, teen angst, slaughter, dark comedy...","[Ross Noble, Tommy Knight, Gemma-Leah Devereux]",[Conor McMahon]
4694,13363,The Man from Earth,An impromptu goodbye party for Professor John ...,"[ScienceFiction, Drama]","[philosophy, secret, birthday, professor, psyc...","[David Lee Smith, Tony Todd, John Billingsley]",[Richard Schenkman]
3683,64720,Take Shelter,"Plagued by a series of apocalyptic visions, a ...","[Thriller, Drama]","[shelter, end of the world, sanity, apocalypse]","[Michael Shannon, Jessica Chastain, Katy Mixon]",[Jeff Nichols]
1891,9469,He Got Game,A basketball player's father must try to convi...,[Drama],"[prison, father son relationship, homicide, be...","[Denzel Washington, Ray Allen, Milla Jovovich]",[Spike Lee]


In [24]:
df_dataset_valid_vector = df_dataset_valid.copy()

In [25]:
df_dataset_valid_vector.loc[:, 'overview'] = df_dataset_valid_vector['overview'].apply(lambda x: x.split())

def remove_space(string_list: list) -> list:
    return [x.replace(" ", "") for x in string_list]

df_dataset_valid_vector.loc[:, 'keywords'] = df_dataset_valid_vector['keywords'].apply(lambda x: remove_space(x))
df_dataset_valid_vector.loc[:, 'cast'] = df_dataset_valid_vector['cast'].apply(lambda x: remove_space(x))
df_dataset_valid_vector.loc[:, 'crew'] = df_dataset_valid_vector['crew'].apply(lambda x: remove_space(x))

In [26]:
# concat columns
df_dataset_valid_vector["tags"] = df_dataset_valid_vector["overview"] + df_dataset_valid_vector["genres"] + df_dataset_valid_vector["keywords"] + df_dataset_valid_vector["cast"] + df_dataset_valid_vector["crew"]

In [27]:
df_processed = df_dataset_valid_vector[['movie_id', 'title', 'tags']]
df_processed["tags"] = df_processed["tags"].apply(lambda x: " ".join(x).lower())

/tmp/ipykernel_100469/2439422044.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed["tags"] = df_processed["tags"].apply(lambda x: " ".join(x).lower())


In [28]:
df_processed.sample(5, random_state=1)

,movie_id,title,tags
3229,20360,Namastey London,indian-born manmohan malhotra decided to re-lo...
4143,115210,Stitches,"the clumsy and unfunny clown richard ""stitches..."
4694,13363,The Man from Earth,an impromptu goodbye party for professor john ...
3683,64720,Take Shelter,"plagued by a series of apocalyptic visions, a ..."
1891,9469,He Got Game,a basketball player's father must try to convi...


In [29]:
import nltk
from nltk.stem import PorterStemmer

In [30]:
ps = PorterStemmer()

In [31]:
def stems(text):
    T = []
    for i in text.split():
        T.append(ps.stem(i))
    
    return " ".join(T)

In [32]:
df_processed["tags"] = df_processed["tags"].apply(stems)

/tmp/ipykernel_100469/769304281.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed["tags"] = df_processed["tags"].apply(stems)


In [33]:
df_processed["tags"]

0       in the 22nd century, a parapleg marin is dispa...
1       captain barbossa, long believ to be dead, ha c...
2       a cryptic messag from bond’ past send him on a...
3       follow the death of district attorney harvey d...
4       john carter is a war-weary, former militari ca...
                              ...                        
4804    el mariachi just want to play hi guitar and ca...
4805    a newlyw couple' honeymoon is upend by the arr...
4806    "signed, sealed, delivered" introduc a dedic q...
4807    when ambiti new york attorney sam is sent to s...
4808    ever sinc the second grade when he first saw h...
Name: tags, Length: 4806, dtype: object

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [35]:
vector = cv.fit_transform(df_processed['tags']).toarray()

In [36]:
from sklearn.metrics.pairwise import cosine_similarity

In [37]:
similarity = cosine_similarity(vector)

In [39]:
def recommend(movie):
    index = df_processed[df_processed['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(df_processed.iloc[i[0]].title)

In [40]:
recommend('Spider-Man 2')

Spider-Man 3
Spider-Man
The Amazing Spider-Man
Iron Man 2
Superman
